In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load and prepare the dataset
file_path = 'goal_ticks.csv'

# Define column names based on the provided structure
column_names = [
    "kart_ID", "ball_X", "ball_Z", "ball_aim_X", "ball_aim_Z", "previous_X", "previous_Z",
    "dist_to_ball", "kart_X", "kart_Z", "vel_X", "vel_Z", "speed", "steer", "accel", "brake", "skid", "goal"
]

df = pd.read_csv(file_path, header=None, names=column_names)

# Extract features and labels
X = df.iloc[:, :-1].values  # All columns except the last one
y = df['goal'].values  # The 'goal' column as binary

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X[:, 1:])  

# Concatenate kart_ID with scaled features
X_with_kartID = torch.tensor(X_scaled, dtype=torch.float32)
kart_ID = torch.tensor(X[:, 0], dtype=torch.float32).unsqueeze(1)
X_with_kartID = torch.cat((kart_ID, X_with_kartID), dim=1)

# Convert to PyTorch tensors
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # Keeping the target as a 2D tensor for BCELoss

# Define the neural network architecture
class ScoringModel(nn.Module):
    def __init__(self, num_inputs=17):
        super(ScoringModel, self).__init__()
        self.layer1 = nn.Linear(num_inputs, 512)
        self.layer2 = nn.Linear(512, 256)
        self.layer3 = nn.Linear(256, 128)
        self.layer4 = nn.Linear(128, 64)
        self.layer5 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)  # Binary classification
        self.leaky_relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.leaky_relu(self.layer1(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer2(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer3(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer4(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer5(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x

# Initialize the model with 17 inputs
model = ScoringModel(num_inputs=17)

# Set up the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_with_kartID, y_tensor, test_size=0.2, random_state=42)

# Prepare data loaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Training function
def train_model(train_loader, model, criterion, optimizer, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        if (epoch + 1) % 1 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Validation function
def validate_model(val_loader, model, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    print(f'Validation Loss: {total_loss / len(val_loader):.4f}')

# Train and validate the model
train_model(train_loader, model, criterion, optimizer, 1)
validate_model(val_loader, model, criterion)

# Save the trained model for deployment
model.eval()
example_input = torch.rand(1, X_train.shape[1])  # Generate a random example input
traced_script_module = torch.jit.trace(model, example_input)
traced_script_module.save("soccer_ai_model.pt")


Epoch 1/250, Loss: 0.0366
Epoch 2/250, Loss: 0.0208
Epoch 3/250, Loss: 0.0190
Epoch 4/250, Loss: 0.1073
Epoch 5/250, Loss: 0.1031
Epoch 6/250, Loss: 0.1154
Epoch 7/250, Loss: 0.2114
Epoch 8/250, Loss: 0.1138
Epoch 9/250, Loss: 0.0857
Epoch 10/250, Loss: 0.0269
Epoch 11/250, Loss: 0.1960
Epoch 12/250, Loss: 0.0898
Epoch 13/250, Loss: 0.0281
Epoch 14/250, Loss: 0.1150
Epoch 15/250, Loss: 0.0555
Epoch 16/250, Loss: 0.2934
Epoch 17/250, Loss: 0.1109
Epoch 18/250, Loss: 0.0691
Epoch 19/250, Loss: 0.0201
Epoch 20/250, Loss: 0.0259
Epoch 21/250, Loss: 0.0251
Epoch 22/250, Loss: 0.1019
Epoch 23/250, Loss: 0.0731
Epoch 24/250, Loss: 0.2874
Epoch 25/250, Loss: 0.3333
Epoch 26/250, Loss: 0.0311
Epoch 27/250, Loss: 0.1024
Epoch 28/250, Loss: 0.1195
Epoch 29/250, Loss: 0.3526
Epoch 30/250, Loss: 0.2184
Epoch 31/250, Loss: 0.3291
Epoch 32/250, Loss: 0.0744
Epoch 33/250, Loss: 0.0445
Epoch 34/250, Loss: 0.0891
Epoch 35/250, Loss: 0.0242
Epoch 36/250, Loss: 0.1718
Epoch 37/250, Loss: 0.0775
Epoch 38/2